In [ ]:
import os
from pathlib import Path

try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
    from google.colab import userdata
    repo_name = 'dgcnz/dl2'
    url = f"https://{userdata.get('gh_pat')}@github.com/{repo_name}.git"
    !git clone {url}
    print("\nCurrent Directory:")
    %cd dl2
    #!pip install torch torchvision numpy matplotlib git+https://github.com/AMLab-Amsterdam/lie_learn escnn scipy
    !pip install torchvision git+https://github.com/AMLab-Amsterdam/lie_learn escnn lightning wandb
    #!pip install -r requirements.txt


else: # automatically checks if the current directory is 'repo name'
    curdir = Path.cwd()
    print("Current Directory", curdir)
    assert curdir.name == "dl2" or curdir.parent.name == "dl2", "Notebook cwd has to be on the project root"
    if curdir.name == "notebooks":
        %cd ..
        print("New Current Directory:", curdir)

Cloning into 'dl2'...
remote: Enumerating objects: 200, done.
remote: Counting objects: 100% (200/200), done.
remote: Compressing objects: 100% (149/149), done.
remote: Total 200 (delta 73), reused 139 (delta 40), pack-reused 0
Receiving objects: 100% (200/200), 92.30 KiB | 794.00 KiB/s, done.
Resolving deltas: 100% (73/73), done.

Current Directory:
/content/dl2
  Cloning https://github.com/AMLab-Amsterdam/lie_learn to /tmp/pip-req-build-15hr9y3b
  Running command git clone --filter=blob:none --quiet https://github.com/AMLab-Amsterdam/lie_learn /tmp/pip-req-build-15hr9y3b
  Resolved https://github.com/AMLab-Amsterdam/lie_learn to commit 1ccc2106e402d517a29de5438c9367c959e67338
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.9/373.9 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0

In [ ]:
import wandb
wandb.login(key=userdata.get('wandb_key'))

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
wandb.init(settings=wandb.Settings(start_method="fork"))

wandb: Currently logged in as: nestam (uva-dl2). Use `wandb login --relogin` to force relogin


In [ ]:
import sys
sys.path.append('../')

import torch

from escnn import gspaces
from escnn import nn
import os
from torch import optim, utils, Tensor
#from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
import lightning as L
from lightning.pytorch.loggers import WandbLogger

from src.models.image_module import ImageLightningModule
from src.data.rotated_mnist_datamodule import MnistRotDataset

from torch.utils.data import Dataset
from torchvision.transforms import RandomRotation
from torchvision.transforms import Pad
from torchvision.transforms import Resize
from torchvision.transforms import ToTensor
from torchvision.transforms import Compose
from torchvision.transforms import InterpolationMode

from torchmetrics import MaxMetric, MeanMetric
from torchmetrics.classification.accuracy import Accuracy

from typing import Any, Dict, Tuple

device = 'cuda' if torch.cuda.is_available() else 'cpu'

###Set up pytorch lightning stuff


In [ ]:
# define the LightningModule
class PlModule(L.LightningModule):
    def __init__(self, net):
        super().__init__()
        self.net = net

        # num classes
        num_classes = 10 #net.layers[-1].out_features #if this break hardcode to one

        # metric objects for calculating and averaging accuracy across batches
        self.train_acc = Accuracy(task="multiclass", num_classes=num_classes)
        self.val_acc = Accuracy(task="multiclass", num_classes=num_classes)
        self.test_acc = Accuracy(task="multiclass", num_classes=num_classes)

        # for averaging loss across batches
        self.train_loss = MeanMetric()
        self.val_loss = MeanMetric()
        self.test_loss = MeanMetric()

        # for tracking best so far validation accuracy
        self.val_acc_best = MaxMetric()

    def model_step(self, batch: Tuple[torch.Tensor, torch.Tensor]) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        """Perform a single model step on a batch of data.

        :param batch: A batch of data (a tuple) containing the input tensor of images and target labels.

        :return: A tuple containing (in order):
            - A tensor of losses.
            - A tensor of predictions.
            - A tensor of target labels.
        """
        x, y = batch
        logits = self.net(x)
        loss =  torch.nn.functional.cross_entropy(logits, y)
        preds = torch.argmax(logits, dim=1)
        return loss, preds, y

    def test_step(self, batch: Tuple[torch.Tensor, torch.Tensor], batch_idx: int) -> None:
      """Perform a single test step on a batch of data from the test set.

      :param batch: A batch of data (a tuple) containing the input tensor of images and target
          labels.
      :param batch_idx: The index of the current batch.
      """
      loss, preds, targets = self.model_step(batch)

      # update and log metrics
      self.test_loss(loss)
      self.test_acc(preds, targets)
      self.log("test/loss", self.test_loss, on_step=False, on_epoch=True, prog_bar=True)
      self.log("test/acc", self.test_acc, on_step=False, on_epoch=True, prog_bar=True)

    def validation_step(self, batch: Tuple[torch.Tensor, torch.Tensor], batch_idx: int) -> None:
        """Perform a single validation step on a batch of data from the validation set.

        :param batch: A batch of data (a tuple) containing the input tensor of images and target
            labels.
        :param batch_idx: The index of the current batch.
        """
        loss, preds, targets = self.model_step(batch)

        # update and log metrics
        self.val_loss(loss)
        self.val_acc(preds, targets)
        self.log("val/loss", self.val_loss, on_step=False, on_epoch=True, prog_bar=True)
        self.log("val/acc", self.val_acc, on_step=False, on_epoch=True, prog_bar=True)

    def training_step(self, batch: Tuple[torch.Tensor, torch.Tensor], batch_idx: int) -> torch.Tensor:
      """Perform a single training step on a batch of data from the training set.

      :param batch: A batch of data (a tuple) containing the input tensor of images and target
          labels.
      :param batch_idx: The index of the current batch.
      :return: A tensor of losses between model predictions and targets.
      """
      loss, preds, targets = self.model_step(batch)

      # update and log metrics
      self.train_loss(loss)
      self.train_acc(preds, targets)
      self.log("train/loss", self.train_loss, on_step=False, on_epoch=True, prog_bar=True)
      self.log("train/acc", self.train_acc, on_step=False, on_epoch=True, prog_bar=True)

      # return loss or backpropagation will fail
      return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.net.parameters(), lr=5e-5, weight_decay=1e-5)
        return optimizer



###C8 steerable cnn


In [ ]:
class C8SteerableCNN(torch.nn.Module):

    def __init__(self, n_classes=10):

        super(C8SteerableCNN, self).__init__()

        # the model is equivariant under rotations by 45 degrees, modelled by C8
        self.r2_act = gspaces.rot2dOnR2(N=8)

        # the input image is a scalar field, corresponding to the trivial representation
        in_type = nn.FieldType(self.r2_act, [self.r2_act.trivial_repr])

        # we store the input type for wrapping the images into a geometric tensor during the forward pass
        self.input_type = in_type

        # convolution 1
        # first specify the output type of the convolutional layer
        # we choose 24 feature fields, each transforming under the regular representation of C8
        out_type = nn.FieldType(self.r2_act, 24*[self.r2_act.regular_repr])
        self.block1 = nn.SequentialModule(
            nn.MaskModule(in_type, 29, margin=1),
            nn.R2Conv(in_type, out_type, kernel_size=7, padding=1, bias=False),
            nn.InnerBatchNorm(out_type),
            nn.ReLU(out_type, inplace=True)
        )

        # convolution 2
        # the old output type is the input type to the next layer
        in_type = self.block1.out_type
        # the output type of the second convolution layer are 48 regular feature fields of C8
        out_type = nn.FieldType(self.r2_act, 48*[self.r2_act.regular_repr])
        self.block2 = nn.SequentialModule(
            nn.R2Conv(in_type, out_type, kernel_size=5, padding=2, bias=False),
            nn.InnerBatchNorm(out_type),
            nn.ReLU(out_type, inplace=True)
        )
        self.pool1 = nn.SequentialModule(
            nn.PointwiseAvgPoolAntialiased(out_type, sigma=0.66, stride=2)
        )

        # convolution 3
        # the old output type is the input type to the next layer
        in_type = self.block2.out_type
        # the output type of the third convolution layer are 48 regular feature fields of C8
        out_type = nn.FieldType(self.r2_act, 48*[self.r2_act.regular_repr])
        self.block3 = nn.SequentialModule(
            nn.R2Conv(in_type, out_type, kernel_size=5, padding=2, bias=False),
            nn.InnerBatchNorm(out_type),
            nn.ReLU(out_type, inplace=True)
        )

        # convolution 4
        # the old output type is the input type to the next layer
        in_type = self.block3.out_type
        # the output type of the fourth convolution layer are 96 regular feature fields of C8
        out_type = nn.FieldType(self.r2_act, 96*[self.r2_act.regular_repr])
        self.block4 = nn.SequentialModule(
            nn.R2Conv(in_type, out_type, kernel_size=5, padding=2, bias=False),
            nn.InnerBatchNorm(out_type),
            nn.ReLU(out_type, inplace=True)
        )
        self.pool2 = nn.SequentialModule(
            nn.PointwiseAvgPoolAntialiased(out_type, sigma=0.66, stride=2)
        )

        # convolution 5
        # the old output type is the input type to the next layer
        in_type = self.block4.out_type
        # the output type of the fifth convolution layer are 96 regular feature fields of C8
        out_type = nn.FieldType(self.r2_act, 96*[self.r2_act.regular_repr])
        self.block5 = nn.SequentialModule(
            nn.R2Conv(in_type, out_type, kernel_size=5, padding=2, bias=False),
            nn.InnerBatchNorm(out_type),
            nn.ReLU(out_type, inplace=True)
        )

        # convolution 6
        # the old output type is the input type to the next layer
        in_type = self.block5.out_type
        # the output type of the sixth convolution layer are 64 regular feature fields of C8
        out_type = nn.FieldType(self.r2_act, 64*[self.r2_act.regular_repr])
        self.block6 = nn.SequentialModule(
            nn.R2Conv(in_type, out_type, kernel_size=5, padding=1, bias=False),
            nn.InnerBatchNorm(out_type),
            nn.ReLU(out_type, inplace=True)
        )
        self.pool3 = nn.PointwiseAvgPoolAntialiased(out_type, sigma=0.66, stride=1, padding=0)

        self.gpool = nn.GroupPooling(out_type)

        # number of output channels
        c = self.gpool.out_type.size

        # Fully Connected
        self.fully_net = torch.nn.Sequential(
            torch.nn.Linear(c, 64),
            torch.nn.BatchNorm1d(64),
            torch.nn.ELU(inplace=True),
            torch.nn.Linear(64, n_classes),
        )

    def forward(self, input: torch.Tensor):
        # wrap the input tensor in a GeometricTensor
        # (associate it with the input type)
        x = nn.GeometricTensor(input, self.input_type)

        # apply each equivariant block

        # Each layer has an input and an output type
        # A layer takes a GeometricTensor in input.
        # This tensor needs to be associated with the same representation of the layer's input type
        #
        # The Layer outputs a new GeometricTensor, associated with the layer's output type.
        # As a result, consecutive layers need to have matching input/output types
        x = self.block1(x)
        x = self.block2(x)
        x = self.pool1(x)

        x = self.block3(x)
        x = self.block4(x)
        x = self.pool2(x)

        x = self.block5(x)
        x = self.block6(x)

        # pool over the spatial dimensions
        x = self.pool3(x)

        # pool over the group
        x = self.gpool(x)

        # unwrap the output GeometricTensor
        # (take the Pytorch tensor and discard the associated representation)
        x = x.tensor

        # classify with the final fully connected layers)
        x = self.fully_net(x.reshape(x.shape[0], -1))

        return x

In [ ]:
# init model
net = C8SteerableCNN().to(device)
equivariantmodel = PlModule(net)

#define transforms
pad = Pad((0, 0, 1, 1), fill=0)

resize1 = Resize(87)
resize2 = Resize(29)

totensor = ToTensor()

train_transform = Compose([
    pad,
    resize1,
    RandomRotation(180., interpolation=InterpolationMode.BILINEAR, expand=False),
    resize2,
    totensor,
])

test_transform = Compose([
    pad,
    totensor,
])


dataset = MnistRotDataset("data/mnist/",download=True, transform=train_transform)
test_dataset = MnistRotDataset("data/mnist/",download=False,train=False, transform=test_transform)

train_loader1 = utils.data.DataLoader(dataset, batch_size = 64, num_workers=7)
test_loader1 = utils.data.DataLoader(test_dataset, batch_size = 64, num_workers=7)


Download completed.
Unzipping dataset...
Unzip completed.


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 7 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
#create logger
wandb.finish()
c8steerable_logger = WandbLogger(project="C8steerable_rotMNIST", log_model='all', name = 'test')

checkpoint_callback = L.pytorch.callbacks.ModelCheckpoint(every_n_epochs = 2)

# train the model (hint: here are some helpful Trainer arguments for rapid idea iteration)
trainer = L.Trainer(max_epochs=3, logger = c8steerable_logger, callbacks = [checkpoint_callback], check_val_every_n_epoch = 1) #for some reason error if you put max_epoch to 1 idk man
trainer.fit(model=equivariantmodel, train_dataloaders=train_loader1, val_dataloaders=test_loader1) #this is fine because the validation set is not used for anything.



epoch,▁▁▂▂▂▃▃▄▄▄▄▅▅▅▆▆▇▇▇███
train/acc,▁▇▇▇▇▇▇█▇███████████
train/loss,█▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
trainer/global_step,▁▁▂▂▂▃▃▄▄▄▄▅▅▅▆▆▇▇▇███
val/acc,▁█
val/loss,█▁
epoch,19
train/acc,0.95725
train/loss,0.13487
trainer/global_step,3759
val/acc,0.93472


INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name         | Type               | Params
----------------------------------------------------
0 | net          | C8SteerableCNN     | 2.1 M 
1 | train_acc    | MulticlassAccuracy | 0     
2 | val_acc      | MulticlassAccuracy | 0     
3 | test_acc     | MulticlassAccuracy | 0     
4 | train_loss   | MeanMetric         | 0     
5 | val_loss     | MeanMetric         | 0     
6 | test_loss    | MeanMetric         | 0     
7 | val_acc_best | MaxMetric          | 0     
----------------------------------------------------
2.1 M     Trainable params
841       Non-trainable params
2.1 M     Total params
8.273     Total estimated model params size (MB)
INFO:lightning.pytorch.callbacks.model_summary:
  | Name         | Type               | Params
----------------------------------------------------
0 | net          | C8SteerableCNN     | 2.1 M 
1 | trai

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


...

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=3` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=3` reached.


###Now create a similar sized non-equivariant CNN


In [ ]:
import torch
import torch.nn.functional as F

class BasicInvertedBottleneckBlock(torch.nn.Module):
    def __init__(self, Cin, N, Cout, downsample = False, first_block = False):
        super(BasicInvertedBottleneckBlock, self).__init__()

        kernel_size = 3
        padding = 1

        if first_block:
          kernel_size = 7
          padding = 3

        self.block = torch.nn.Sequential(
          torch.nn.Conv2d(Cin, N, kernel_size=kernel_size, stride=1, padding=padding),
          torch.nn.BatchNorm2d(N), #idk which BN to use
          torch.nn.ELU(inplace=True),
          #dit is de enige logische interpretatie van appendix H4 die ik kan bedenken. of miss dit of downsample maar nooit beide? dat is denk ik logischer
          #torch.nn.Conv2d(N, Cout, kernel_size=kernel_size, stride=1, padding=padding)
        )

        self.one_by_one: bool = (Cin != Cout)

        if self.one_by_one:
            self.conv1x1 = torch.nn.Conv2d(Cin, Cout, kernel_size=1, stride=1, padding=0)

        self.downsample = downsample
        if self.downsample:
          #for this one I'm guessing
          self.second_conv = torch.nn.Conv2d(N, Cout, kernel_size=kernel_size, stride=2, padding=padding)
          self.avg_pool = torch.nn.AvgPool2d(kernel_size=kernel_size, stride=2, padding=padding)
        else:
          self.second_conv = torch.nn.Conv2d(N, Cout, kernel_size=kernel_size, stride=1, padding=padding) #idk setting kernel size to one here does massively reduce the number of parameters

    def forward(self, x):
        out = self.block(x)
        print('before downsample')
        print(out.shape)
        print(x.shape)

        out = self.second_conv(out)
        if self.downsample:
          #print('downsampled')
          x = self.avg_pool(x)


        print('afterdownsample')
        print(out.shape)
        print(x.shape)

        if self.one_by_one:
            x = self.conv1x1(x)

        print('afteronebyone')
        print(out.shape)
        print(x.shape)
        skip_connection = out + x
        return skip_connection

class CNN(torch.nn.Module):
    def __init__(self, backbone_channels, residual_channels, n_classes):
        super(CNN, self).__init__()
        self.backbone_channels = backbone_channels
        self.residual_channels = residual_channels
        self.blocks = self._make_blocks()
        self.max_pool = torch.nn.MaxPool2d(kernel_size=3)

        # Fully Connected
        self.fully_net = torch.nn.Sequential(
            torch.nn.Linear(128, 64),
            torch.nn.BatchNorm1d(64),
            torch.nn.ELU(inplace=True),
            torch.nn.Linear(64, n_classes),
        )

    def _make_blocks(self):
        blocks = []
        for i in range(len(self.backbone_channels)):
            Cin = self.backbone_channels[i]
            N = self.residual_channels[i]
            # this next part is because their explanation doesnt seem to include the channel output size of the last block. So we set it to be the same as the input
            if i < len(self.backbone_channels) - 1:
                Cout = self.backbone_channels[i + 1]
            else:
                Cout = 128 #this is also very vague from the og paper

            #print('hello', i)
            if i == 0:
              #print('i = 0 -------------------------')
              blocks.append(BasicInvertedBottleneckBlock(Cin, N, Cout, first_block=True))

            elif (i+1) % 2 == 0: #every two layers this is true
              #print('True-------------------------------------------------------------')
              blocks.append(BasicInvertedBottleneckBlock(Cin, N, Cout, downsample=True))

            else:
              #print('normal case =======================')
              blocks.append(BasicInvertedBottleneckBlock(Cin, N, Cout))
        return torch.nn.Sequential(*blocks)

    def forward(self, x):
        out = self.blocks(x)
        out = self.max_pool(out)
        print(out.shape)
        out = self.fully_net(out.flatten(start_dim = 1))

        return out

# Example usage:
backbone_channels = [1, 21, 54, 72, 108, 168]  # These are the C_in's
residual_channels = [96, 192, 288, 288, 576, 576]  # These are the upsampled N's
model = CNN(backbone_channels, residual_channels, n_classes = 10)



In [ ]:
import torch
import torch.nn.functional as F
#V2

class BasicInvertedBottleneckBlockV2(torch.nn.Module):
    def __init__(self, Cin, N, Cout, downsample = False, first_block = False):
        super(BasicInvertedBottleneckBlockV2, self).__init__()

        kernel_size = 3
        padding = 1

        if first_block:
          kernel_size = 7
          padding = 3

        self.block = torch.nn.Sequential(
          torch.nn.Conv2d(Cin, N, kernel_size=kernel_size, stride=1, padding=padding),
          torch.nn.BatchNorm2d(N), #idk which BN to use
          torch.nn.ELU(inplace=True),
          #dit is de enige logische interpretatie van appendix H4 die ik kan bedenken. of miss dit of downsample maar nooit beide? dat is denk ik logischer
          #torch.nn.Conv2d(N, Cout, kernel_size=kernel_size, stride=1, padding=padding)
        )

        self.one_by_one: bool = (Cin != Cout)

        if self.one_by_one:
            self.conv1x1 = torch.nn.Conv2d(Cin, Cout, kernel_size=1, stride=1, padding=0)

        self.downsample = downsample
        if self.downsample:
          #for this one I'm guessing
          self.second_conv = torch.nn.Conv2d(N, Cin, kernel_size=kernel_size, stride=2, padding=padding)
          self.avg_pool = torch.nn.AvgPool2d(kernel_size=kernel_size, stride=2, padding=padding)
        else:
          self.second_conv = torch.nn.Conv2d(N, Cin, kernel_size=kernel_size, stride=1, padding=padding) #idk setting kernel size to one here does massively reduce the number of parameters

    def forward(self, x):
        out = self.block(x)
        #N Channels
        print('before downsample')
        print(out.shape)
        print(x.shape)

        out = self.second_conv(out)
        #C_in Channels

        if self.downsample:
          #if the second convolution downsamples
          #then the size of the image changes
          #so we must decrease x in size
          #to be able to add it up
          x = self.avg_pool(x)


        print('afterdownsample')
        print(out.shape)
        print(x.shape)




        print('afteronebyone')
        print(out.shape)
        print(x.shape)
        skip_connection = out + x
        if self.one_by_one:
           skip_connection = self.conv1x1(skip_connection)
           #C_out

        return skip_connection

class CNN(torch.nn.Module):
    def __init__(self, backbone_channels, residual_channels, n_classes):
        super(CNN, self).__init__()
        self.backbone_channels = backbone_channels
        self.residual_channels = residual_channels
        self.blocks = self._make_blocks()
        self.max_pool = torch.nn.MaxPool2d(kernel_size=3)

        # Fully Connected
        self.fully_net = torch.nn.Sequential(
            torch.nn.Linear(128, 64),
            torch.nn.BatchNorm1d(64),
            torch.nn.ELU(inplace=True),
            torch.nn.Linear(64, n_classes),
        )

    def _make_blocks(self):
        blocks = []
        for i in range(len(self.backbone_channels)):
            Cin = self.backbone_channels[i]
            N = self.residual_channels[i]
            # this next part is because their explanation doesnt seem to include the channel output size of the last block. So we set it to be the same as the input
            if i < len(self.backbone_channels) - 1:
                Cout = self.backbone_channels[i + 1]
            else:
                Cout = 128 #this is also very vague from the og paper

            #print('hello', i)
            if i == 0:
              #print('i = 0 -------------------------')
              blocks.append(BasicInvertedBottleneckBlockV2(Cin, N, Cout, first_block=True))

            elif (i+1) % 2 == 0: #every two layers this is true
              #print('True-------------------------------------------------------------')
              blocks.append(BasicInvertedBottleneckBlockV2(Cin, N, Cout, downsample=True))

            else:
              #print('normal case =======================')
              blocks.append(BasicInvertedBottleneckBlockV2(Cin, N, Cout))
        return torch.nn.Sequential(*blocks)

    def forward(self, x):
        out = self.blocks(x)
        out = self.max_pool(out)
        print(out.shape)
        out = self.fully_net(out.flatten(start_dim = 1))

        return out

# Example usage:
backbone_channels = [1, 21, 54, 72, 108, 168]  # These are the C_in's
residual_channels = [96, 192, 288, 288, 576, 576]  # These are the upsampled N's
model = CNN(backbone_channels, residual_channels, n_classes = 10)



In [ ]:
normal_cnn = PlModule(model)

train_loader2 = utils.data.DataLoader(dataset, batch_size = 64, num_workers=7)

#create logger
cnn_logger = WandbLogger(project="CNN_rotMNIST", log_model='all')

cnn_trainer = L.Trainer(limit_train_batches=1, max_epochs=2, logger = cnn_logger) #for some reason error if you put max_epoch to 1 idk man
cnn_trainer.fit(model=normal_cnn, train_dataloaders=train_loader2)



/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 7 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: `Trainer(limit_trai

Training: |          | 0/? [00:00<?, ?it/s]

before downsample
torch.Size([64, 96, 29, 29])
torch.Size([64, 1, 29, 29])
afterdownsample
torch.Size([64, 1, 29, 29])
torch.Size([64, 1, 29, 29])
afteronebyone
torch.Size([64, 1, 29, 29])
torch.Size([64, 1, 29, 29])
before downsample
torch.Size([64, 192, 29, 29])
torch.Size([64, 21, 29, 29])
afterdownsample
torch.Size([64, 21, 15, 15])
torch.Size([64, 21, 15, 15])
afteronebyone
torch.Size([64, 21, 15, 15])
torch.Size([64, 21, 15, 15])
before downsample
torch.Size([64, 288, 15, 15])
torch.Size([64, 54, 15, 15])
afterdownsample
torch.Size([64, 54, 15, 15])
torch.Size([64, 54, 15, 15])
afteronebyone
torch.Size([64, 54, 15, 15])
torch.Size([64, 54, 15, 15])
before downsample
torch.Size([64, 288, 15, 15])
torch.Size([64, 72, 15, 15])
afterdownsample
torch.Size([64, 72, 8, 8])
torch.Size([64, 72, 8, 8])
afteronebyone
torch.Size([64, 72, 8, 8])
torch.Size([64, 72, 8, 8])
before downsample
torch.Size([64, 576, 8, 8])
torch.Size([64, 108, 8, 8])
afterdownsample
torch.Size([64, 108, 8, 8])
torc

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


before downsample
torch.Size([64, 96, 29, 29])
torch.Size([64, 1, 29, 29])
afterdownsample
torch.Size([64, 1, 29, 29])
torch.Size([64, 1, 29, 29])
afteronebyone
torch.Size([64, 1, 29, 29])
torch.Size([64, 1, 29, 29])
before downsample
torch.Size([64, 192, 29, 29])
torch.Size([64, 21, 29, 29])
afterdownsample
torch.Size([64, 21, 15, 15])
torch.Size([64, 21, 15, 15])
afteronebyone
torch.Size([64, 21, 15, 15])
torch.Size([64, 21, 15, 15])
before downsample
torch.Size([64, 288, 15, 15])
torch.Size([64, 54, 15, 15])
afterdownsample
torch.Size([64, 54, 15, 15])
torch.Size([64, 54, 15, 15])
afteronebyone
torch.Size([64, 54, 15, 15])
torch.Size([64, 54, 15, 15])
before downsample
torch.Size([64, 288, 15, 15])
torch.Size([64, 72, 15, 15])
afterdownsample
torch.Size([64, 72, 8, 8])
torch.Size([64, 72, 8, 8])
afteronebyone
torch.Size([64, 72, 8, 8])
torch.Size([64, 72, 8, 8])
before downsample
torch.Size([64, 576, 8, 8])
torch.Size([64, 108, 8, 8])
afterdownsample
torch.Size([64, 108, 8, 8])
torc

INFO: `Trainer.fit` stopped: `max_epochs=2` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=2` reached.
